# Install Packages

In [ ]:
# Install torch metrics package for LOSS, EVALUATION METRICS
!pip install lightning-utilities
!pip install torchmetrics --no-deps
!pip install torchinfo
!pip install -U albumentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 21.0 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix


import torch
import torch.nn as nn
import torchvision

from torchvision.transforms import v2
from torchvision.transforms.functional import to_pil_image

from torchmetrics import MeanMetric, Accuracy
from torchmetrics import ConfusionMatrix, Precision, Recall, F1Score
from torchinfo import summary

import torch
from torch.utils.data import random_split, DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
import albumentations as alb
from albumentations.pytorch import ToTensorV2
import gc
import copy

In [ ]:
print(torch.cuda.get_device_name() if torch.cuda.is_available() else "No GPU detected")


Tesla T4


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print("Device:", device)

Device: cuda


#Albumentations

resizing to 384

In [ ]:
import albumentations as alb
from albumentations.pytorch import ToTensorV2

train_augmentations = {
    "DF": alb.Compose([
        alb.HorizontalFlip(p=0.7),
        alb.Rotate(limit=270, p=0.7),
        alb.RandomBrightnessContrast(p=0.7),
        alb.CoarseDropout(max_holes=2, max_height=18, max_width=18, p=0.3),
        alb.GaussianBlur(p=0.2),
        alb.Resize(384,384),
        alb.Normalize(mean=[0.485, 0.456, 0.406],
                  std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]),
    "VASC": alb.Compose([
        alb.HorizontalFlip(p=0.7),
        alb.Rotate(limit=270, p=0.7),
        alb.RandomBrightnessContrast(p=0.7),
        alb.CoarseDropout(max_holes=2, max_height=18, max_width=18, p=0.3),
        alb.GaussianBlur(p=0.2),
        alb.Resize(384,384),
        alb.Normalize(mean=[0.485, 0.456, 0.406],
                  std=[0.229, 0.224, 0.225]),
        ToTensorV2()

    ]),
    "AK": alb.Compose([
        alb.HorizontalFlip(p=0.7),
        alb.Rotate(limit=270, p=0.7),
        alb.RandomBrightnessContrast(p=0.7),
        alb.CoarseDropout(max_holes=2, max_height=18, max_width=18, p=0.3),
        alb.Resize(384,384),
        alb.Normalize(mean=[0.485, 0.456, 0.406],
                  std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]),
    "SCC": alb.Compose([
        alb.HorizontalFlip(p=0.7),
        alb.Rotate(limit=270, p=0.7),
        alb.RandomBrightnessContrast(p=0.3),
        alb.CoarseDropout(max_holes=2, max_height=18, max_width=18, p=0.3),
        alb.GaussianBlur(p=0.2),
        alb.Resize(384,384),
        alb.Normalize(mean=[0.485, 0.456, 0.406],
                  std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]),
    "BKL":alb.Compose([
        alb.HorizontalFlip(p=0.5),
        alb.Rotate(limit=270, p=0.5),
        alb.RandomBrightnessContrast(p=0.5),
        alb.Resize(384,384),
        alb.Normalize(mean=[0.485, 0.456, 0.406],
                  std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]),
    "BCC":alb.Compose([
        alb.HorizontalFlip(p=0.3),
        alb.Rotate(limit=270, p=0.5),
        alb.RandomBrightnessContrast(p=0.5),
        alb.Resize(384,384),
        alb.Normalize(mean=[0.485, 0.456, 0.406],
                  std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]),
    "MEL":alb.Compose([
        alb.HorizontalFlip(p=0.3),
        alb.RandomBrightnessContrast(p=0.4),
        alb.Resize(384,384),
        alb.Normalize(mean=[0.485, 0.456, 0.406],
                  std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]),
    "NV":alb.Compose([
        alb.Resize(384,384),
        alb.Normalize(mean=[0.485, 0.456, 0.406],
                  std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])
}

/tmp/ipython-input-673664449.py:9: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  alb.CoarseDropout(max_holes=2, max_height=18, max_width=18, p=0.3),
/tmp/ipython-input-673664449.py:20: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  alb.CoarseDropout(max_holes=2, max_height=18, max_width=18, p=0.3),
/tmp/ipython-input-673664449.py:32: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  alb.CoarseDropout(max_holes=2, max_height=18, max_width=18, p=0.3),
/tmp/ipython-input-673664449.py:42: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  alb.CoarseDropout(max_holes=2, max_height=18, max_width=18, p=0.3),


In [ ]:
class AlbTransform:
    def __init__(self, augmentation_dict):
        self.augmentation_dict = augmentation_dict

    def __call__(self, img):
        img = np.array(img)  # Convert PIL Image to NumPy array
        label = img.filename.split("/")[-2]  # Extract label from folder structure

        if label in self.augmentation_dict:
            augmented = self.augmentation_dict[label](image=img)["image"]
        else:
            augmented = img  # No augmentation applied

        tensor_img = ToTensorV2()(image=augmented)["image"]
        tensor_img = tensor_img.permute(2, 0, 1).contiguous()  # Fix channel order

        return tensor_img


# Train/Val Set

In [ ]:
from torch.utils.data import Subset
from collections import Counter
import torch
from sklearn.model_selection import StratifiedKFold

# Load entire dataset once
dataset = ImageFolder(root="/content/drive/MyDrive/ISIC_2019_raw", transform=None)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=27)

for fold, (train_idx, val_idx) in enumerate(skf.split(np.arange(len(dataset)), dataset.targets)):
    print(f"\n===== Fold {fold+1} =====")
    train_set = Subset(dataset, train_idx)
    val_set   = Subset(dataset, val_idx)




===== Fold 1 =====

===== Fold 2 =====

===== Fold 3 =====

===== Fold 4 =====

===== Fold 5 =====


In [ ]:
 # Define classes
classes = ["AK", "BCC", "BKL","DF","MEL","NV","SCC","VASC"]
num_class=len(classes)
print("Number of Classes:", num_class)

Number of Classes: 8


In [ ]:
class_counts = Counter([dataset.targets[i] for i in train_idx])
weights = np.array([1.0 / class_counts[c] for c in range(len(classes))], dtype=np.float32)
class_weights_tensor = torch.tensor(weights, dtype=torch.float32).to(device)

In [ ]:
# Sampler
from torch.utils.data import WeightedRandomSampler
sample_weights = torch.tensor([weights[dataset.targets[i]] for i in train_idx],dtype=torch.float)
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

In [ ]:
train_set.dataset.transform = AlbTransform(train_augmentations)  # Augmentations only for training
val_set.dataset.transform = transforms.Compose([
    transforms.Resize((384,384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])  #  No augmentations for validation


In [ ]:
test_set=ImageFolder(root="/content/drive/MyDrive/ISIC_2019_test_raw", transform=transforms.Compose([
    transforms.Resize((384,384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]))

reduced from 64 as increased computational power for resizing to 384x384

In [ ]:
train_loader = DataLoader(train_set, batch_size=8, sampler=sampler)
val_loader = DataLoader(val_set, batch_size=8, shuffle=False)
test_loader=DataLoader(test_set, batch_size=8, shuffle=False)

In [ ]:
print("Number of Train Samples:", len(train_set))
print("Number of Validation Samples:", len(val_set))
print("Number of Test Samples:", len(test_set))


Number of Train Samples: 20265
Number of Validation Samples: 5066
Number of Test Samples: 8238


In [ ]:
 # Define classes
classes = ["AK", "BCC", "BKL","DF","MEL","NV","SCC","VASC"]
num_class=len(classes)
print("Number of Classes:", num_class)

Number of Classes: 8


#Pretrained Models

Next steps
1. Load Pretrained Models
2. Maybe make my own?/Contrastive Learning Model and compare
    - MoCo (classifier fine tune)
    - would need contrastive learning loss function for pretraining
    - Compare contrastive embeddings vs CNN

3. Evaluate Metrics F1, acc, prec, auc
4. Grad-CAM/SHAP interpretability inspected

5. Fusion Model using patient metadata
6. Evaluation metrics F1, acc,prec, auc
7. Grad-CAM/SHAP intrepretability inspected

8. Compare Overall results
  - Img ONLY vs Img + Metadata:
    - F1 scores, reliability (precision)
    - Interpretability (heatmap)
    - Interpretability (SHAP)



In [ ]:
import torchvision.models as models

In [ ]:
project_root = "/content/drive/MyDrive/Thesis/Updated MODELS"
os.makedirs(project_root, exist_ok=True)

In [ ]:
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecision, MulticlassRecall, MulticlassF1Score, MulticlassAUROC

## Resnet

In [ ]:
model_name = "ResNet50"
checkpoint_path = os.path.join(project_root, f"{model_name}_checkpoint_latest.pth")
train_val_path = os.path.join(project_root, f"{model_name}_train_val_results.csv")
test_results_path = os.path.join(project_root, f"{model_name}_test_results.csv")

In [ ]:
res_model18=models.resnet18(pretrained=True)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 51.6MB/s]


In [ ]:
res_model18.fc= nn.Sequential(
    nn.Dropout(p=0.5), # Look at changing to 0.6
    nn.Linear(res_model18.fc.in_features, num_class))

In [ ]:
res_model50 = models.resnet50(pretrained=True)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 175MB/s]


In [ ]:
res_model50.fc= nn.Sequential(
    nn.Dropout(p=0.5), # Look at changing to 0.6
    nn.Linear(res_model50.fc.in_features, num_class))

## Vision Transformers

used for medical imaging
- robust to dataset imbalance

In [ ]:
import timm
vis_model = timm.create_model("vit_base_patch16_224", pretrained=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
print(vis_model)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False

In [ ]:
vis_model.head = nn.Linear(vis_model.head.in_features, num_class)

## Efficient Net

excellent accuracy balance depth width and resolution

In [ ]:
import timm
eff_model = timm.create_model("efficientnet_b3",pretrained=True)

model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]

In [ ]:
eff_model.classifier =nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(eff_model.classifier.in_features, num_class))

# Models running

In [ ]:
device = 'cuda' if torch.cuda.is_available()\
          else 'mps' if torch.mps.is_available()\
          else 'cpu'
print('device', device)

device cuda


In [ ]:
!pip install focal-loss

In [ ]:
!pip show focal-loss

Name: focal-loss
Version: 0.0.7
Summary: TensorFlow implementation of focal loss.
Home-page: https://github.com/artemmavrin/focal-loss
Author: Artem Mavrin
Author-email: artemvmavrin@gmail.com
License: Apache 2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: tensorflow
Required-by: 


In [ ]:
# As using pytorch need class to use focal loss
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        log_probs = F.log_softmax(inputs, dim=1)
        probs = torch.exp(log_probs)
        targets_one_hot = F.one_hot(targets, num_classes=inputs.size(1)).float()

        focal_term = (1 - probs) ** self.gamma
        loss = -targets_one_hot * focal_term * log_probs

        if self.alpha is not None:
            alpha = torch.tensor(self.alpha).to(inputs.device)
            loss = loss * alpha

        loss = loss.sum(dim=1)

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss


In [ ]:
#criteria = torch.nn.CrossEntropyLoss(label_smoothing=0.1) original attempt
# Initially without class weights seems to be overfittinhg
#class_weights = [class_weights_dict[i] for i in range(num_class)]
#criteria = FocalLoss(gamma=2.0, alpha=class_weights)

# Class weights computed per fold
class_counts = Counter([dataset.targets[i] for i in train_idx])
weights = [1.0 / class_counts[c] for c in range(len(classes))]
weights_tensor = torch.tensor(weights, dtype=torch.float32).to(device)

criteria = FocalLoss(gamma=2.0, alpha=weights_tensor)

#Epoch

In [ ]:
from torchmetrics import Accuracy, Precision, Recall, F1Score, AUROC, ConfusionMatrix, MeanMetric

In [ ]:
from torchmetrics.classification import (
    MulticlassAccuracy,
    MulticlassPrecision,
    MulticlassRecall,
    MulticlassF1Score
)

In [ ]:
from tqdm import tqdm
import time

def one_train_epoch(model, optimizer):
    start_time = time.time()

    # Macro metrics
    losses = MeanMetric().to(device)
    acc = MulticlassAccuracy(num_classes=num_class, average='macro').to(device)
    precision = MulticlassPrecision(num_classes=num_class, average='macro').to(device)
    recall = MulticlassRecall(num_classes=num_class, average='macro').to(device)
    f1_macro = MulticlassF1Score(num_classes=num_class, average='macro').to(device)

    # Per-class F1
    f1_per_class = MulticlassF1Score(num_classes=num_class, average=None).to(device)

    model.train()

    all_preds= []
    all_targets=[]

    for batch_idx, (X, Y) in enumerate(tqdm(train_loader, desc="Training", leave=False)):
        X, Y = X.to(device), Y.to(device)

        optimizer.zero_grad()
        preds = model(X)
        loss = criteria(preds, Y)
        loss.backward()
        optimizer.step()


        losses.update(loss.item(), X.size(0))
        acc.update(preds, Y)
        precision.update(preds, Y)
        recall.update(preds, Y)
        f1_macro.update(preds, Y)

        all_preds.append(preds.argmax(dim=1))
        all_targets.append(Y)

    # Concatenate predictions and targets for per-class F1
    all_preds = torch.cat(all_preds)
    all_targets = torch.cat(all_targets)
    f1_class_scores = f1_per_class(all_preds, all_targets)


    print(f"Train epoch duration: {time.time() - start_time:.2f} seconds")
    for i, score in enumerate(f1_class_scores):
        print(f"Train F1 - Class {i}: {score:.4f}")

    return (
        losses.compute().item(),
        acc.compute().item(),
        precision.compute().item(),
        recall.compute().item(),
        f1_macro.compute().item(),
        f1_class_scores.cpu().numpy()  # Optional: return for logging or plotting
    )

In [ ]:
def one_val_epoch(model):
    start_time = time.time()

    # Macro metrics
    losses = MeanMetric().to(device)
    acc = MulticlassAccuracy(num_classes=num_class, average='macro').to(device)
    precision = MulticlassPrecision(num_classes=num_class, average='macro').to(device)
    recall = MulticlassRecall(num_classes=num_class, average='macro').to(device)
    f1_macro = MulticlassF1Score(num_classes=num_class, average='macro').to(device)
    auc = MulticlassAUROC(num_classes=num_class, average='macro').to(device)

    # Per-class F1
    f1_per_class = MulticlassF1Score(num_classes=num_class, average=None).to(device)

    model.eval()
    model.to(device)

    all_preds=[]
    all_targets=[]

    with torch.no_grad():
        for X, Y in tqdm(val_loader, desc="Validating", leave=False):
            X, Y = X.to(device), Y.to(device)
            preds = model(X)
            loss = criteria(preds, Y)

            losses.update(loss.item(), X.size(0))
            acc.update(preds, Y)
            precision.update(preds, Y)
            recall.update(preds, Y)
            f1_macro.update(preds, Y)
            auc.update(preds, Y)

            all_preds.append(preds.argmax(dim=1))
            all_targets.append(Y)

    # Compute per-class F1
    all_preds = torch.cat(all_preds)
    all_targets = torch.cat(all_targets)
    f1_class_scores = f1_per_class(all_preds, all_targets)

    print(f"Validation took {time.time() - start_time:.2f}s")
    for i, score in enumerate(f1_class_scores):
        print(f"Val F1 - Class {i}: {score:.4f}")

    return (
        losses.compute().item(),
        acc.compute().item(),
        precision.compute().item(),
        recall.compute().item(),
        f1_macro.compute().item(),
        auc.compute().item(),
        f1_class_scores.cpu().numpy()  # Optional: return for logging or plotting
    )


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
def one_test_epoch(model):
    start_time = time.time()

    # Macro metrics
    losses = MeanMetric().to(device)
    acc = MulticlassAccuracy(num_classes=num_class, average='macro').to(device)
    precision = MulticlassPrecision(num_classes=num_class, average='macro').to(device)
    recall = MulticlassRecall(num_classes=num_class, average='macro').to(device)
    f1_macro = MulticlassF1Score(num_classes=num_class, average='macro').to(device)
    auc = MulticlassAUROC(num_classes=num_class, average='macro').to(device)

    # Per-class F1
    f1_per_class = MulticlassF1Score(num_classes=num_class, average=None).to(device)

    model.eval()
    model.to(device)

    true_labels_log = []
    predictions_log = []

    all_preds=[]
    all_targets=[]

    with torch.no_grad():
        for X, Y in tqdm(test_loader, desc="Testing", leave=False):
            X, Y = X.to(device), Y.to(device)
            preds = model(X)
            loss = criteria(preds, Y)

            losses.update(loss.item(), X.size(0))
            acc.update(preds, Y)
            precision.update(preds, Y)
            recall.update(preds, Y)
            f1_macro.update(preds, Y)
            auc.update(preds, Y)

            pred_labels = preds.argmax(dim=1)
            true_labels_log.extend(Y.cpu().numpy())
            predictions_log.extend(pred_labels.cpu().numpy())

            all_preds.append(pred_labels)
            all_targets.append(Y)

    # Compute per-class F1
    all_preds = torch.cat(all_preds)
    all_targets = torch.cat(all_targets)
    f1_class_scores = f1_per_class(all_preds, all_targets)

    print(f"Testing took {time.time() - start_time:.2f}s")
    print("\nDetailed Classification Report:")
    print(classification_report(true_labels_log, predictions_log, digits=4))

    # Save full classification report
    report_dict = classification_report(true_labels_log, predictions_log, digits=4, output_dict=True)
    report_df = pd.DataFrame(report_dict).transpose()
    report_df.to_csv(os.path.join(project_root, f"{model_name}_class_report.csv"))

    # Save per-class metrics only (excluding summary rows)
    per_class_metrics = report_df.drop(['accuracy', 'macro avg', 'weighted avg'], errors='ignore')
    per_class_metrics.to_csv(os.path.join(project_root, f"{model_name}_per_class_metrics.csv"))

    # Optional: Print per-class F1 from torchmetrics
    print("\nTorchMetrics Per-Class F1:")
    for i, score in enumerate(f1_class_scores):
        print(f"Test F1 - Class {i}: {score:.4f}")

    return (
        losses.compute().item(),
        acc.compute().item(),
        precision.compute().item(),
        recall.compute().item(),
        f1_macro.compute().item(),
        auc.compute().item(),
        true_labels_log,
        predictions_log,
        f1_class_scores.cpu().numpy()  # Optional: return for logging or visualization
    )


In [ ]:
train_val_results=[]
def run_training(model_name, model, optimizer, scheduler,
                 train_val_path, test_results_path, checkpoint_path,
                 num_epochs=30, patience=15, min_delta=0.003,
                 minority_classes=["AK", "DF", "SCC", "VASC"]):

    model = model.to(device)
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_f1 = 0.0
    patience_counter = 0
    start_epoch = 0

    # Resume from checkpoint if exists
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        start_epoch = checkpoint.get("epoch", 0) + 1
        print(f"{model_name} Resuming training from epoch {start_epoch}...")

    for epoch in range(start_epoch, num_epochs):
        # === Train & Validate ===
        train_loss, train_acc, train_prec, train_rec, train_f1, train_f1_per_class = one_train_epoch(model, optimizer)
        val_loss, val_acc, val_prec, val_rec, val_f1, val_auc, val_f1_per_class = one_val_epoch(model)

        scheduler.step()
        current_lr = optimizer.param_groups[0]['lr']
        print(f"Current LR: {current_lr:.6f}")

        # === Compute minority macro F1 ===
        minority_idx = [classes.index(c) for c in minority_classes]
        minority_macro_f1 = float(np.mean([val_f1_per_class[i] for i in minority_idx]))

        # === Log metrics ===
        metrics = {
            "Model": model_name, "Epoch": epoch,
            "Train Loss": train_loss, "Train Accuracy": train_acc,
            "Train Precision": train_prec, "Train Recall": train_rec, "Train F1": train_f1,
            "Val Loss": val_loss, "Val Accuracy": val_acc,
            "Val Precision": val_prec, "Val Recall": val_rec, "Val F1": val_f1,
            "Val Minority Macro F1": minority_macro_f1,
            "Val AUC": val_auc
        }
        train_val_results.append(metrics)
        pd.DataFrame([metrics]).to_csv(train_val_path, mode='a',
                                       header=not os.path.exists(train_val_path),
                                       index=False)

        # === Log per-class F1s ===
        f1_df = pd.DataFrame({
            "Epoch": [epoch] * num_class,
            "Class": list(range(num_class)),
            "Train F1": train_f1_per_class,
            "Val F1": val_f1_per_class
        })
        f1_df.to_csv(os.path.join(project_root, f"{model_name}_f1_per_class_by_epoch.csv"),
                     mode='a', header=not os.path.exists(os.path.join(project_root, f"{model_name}_f1_per_class_by_epoch.csv")),
                     index=False)

        print(f"Epoch {epoch+1}/{num_epochs} | Train F1: {train_f1:.4f} | "
              f"Val F1: {val_f1:.4f} | Minority F1: {minority_macro_f1:.4f}")

        # === Early stopping on macro F1 ===
        if val_f1 > best_val_f1 + min_delta:
            best_val_f1 = val_f1
            best_model_wts = copy.deepcopy(model.state_dict())
            patience_counter = 0
            # Save best F1 checkpoint
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'tag': 'best_f1'
            }, os.path.join(project_root, f"{model_name}_best_f1.pth"))
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping activated.")
                break

        # Save periodic checkpoint
        if (epoch + 1) % 5 == 0:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'tag': 'intermediate'
            }, os.path.join(project_root, f"{model_name}_checkpoint_epoch_{epoch}.pth"))

        # Always save latest
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'tag': 'latest'
        }, checkpoint_path)

        gc.collect()

    # === Restore best F1 weights ===
    model.load_state_dict(best_model_wts)
    print(f"{model_name} Training complete. Running test set...")

    # === Test ===
    test_loss, test_acc, test_prec, test_rec, test_f1, test_auc, y_true, y_pred, test_f1_per_class = one_test_epoch(model)

    test_row = {
        "Model": model_name,
        "Test Loss": test_loss, "Test Accuracy": test_acc,
        "Test Precision": test_prec, "Test Recall": test_rec,
        "Test F1": test_f1, "Test AUC": test_auc
    }
    pd.DataFrame([test_row]).to_csv(test_results_path, mode='a',
                                    header=not os.path.exists(test_results_path),
                                    index=False)

    # Save predictions
    pd.DataFrame({"actual": y_true, "predicted": y_pred}).to_csv(
        os.path.join(project_root, f"{model_name}_predictions.csv"), index=False)

    # Save per-class metrics
    report_dict = classification_report(y_true, y_pred, digits=4, output_dict=True)
    per_class_metrics = pd.DataFrame(report_dict).transpose().drop(['accuracy', 'macro avg', 'weighted avg'], errors='ignore')
    per_class_metrics.to_csv(os.path.join(project_root, f"{model_name}_per_class_metrics.csv"))

    # Save torchmetrics per-class F1
    pd.DataFrame({"Class": list(range(num_class)), "Test F1": test_f1_per_class}).to_csv(
        os.path.join(project_root, f"{model_name}_test_f1_per_class.csv"), index=False)

    print("Test results saved, predictions log saved, and per-class metrics exported.")


#Models to train

In [ ]:
def make_resnet18(num_class):
    model = models.resnet18(pretrained=True)
    model.fc = nn.Sequential(
        nn.Dropout(p=0.5),  # you can change to 0.6 if you want more regularisation
        nn.Linear(model.fc.in_features, num_class)
    )
    return model

def make_efficientnet_b3(num_class):
    model = timm.create_model("efficientnet_b3", pretrained=True)
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.5),
        nn.Linear(model.classifier.in_features, num_class)
    )
    return model


In [ ]:
import os
import gc
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset, DataLoader, WeightedRandomSampler
import torchvision.models as models  # for ResNet18 if you re-enable it


# === MoCo (query-only) model ===
class MoCoEffComboModel(nn.Module):
    def __init__(self, encoder_q, encoder_k, queue, embedding_dim, alpha=0.5, num_class=3):
        super().__init__()
        self.encoder_q = encoder_q
        self.encoder_k = encoder_k
        for p in self.encoder_k.parameters():
            p.requires_grad = False
        self.encoder_k.eval()

        # Register queue as a buffer so .to(device) moves it automatically
        self.register_buffer("queue", queue)  # shape: [queue_size, embedding_dim]
        self.alpha = alpha  # kept for compatibility, not used in query-only
        self.classifier = nn.Linear(embedding_dim, num_class)

    def forward(self, x):
        # Query-only path, but still updates the momentum queue
        query_features, _, self.queue = moco_forward(self.encoder_q, self.encoder_k, self.queue, x)

        if query_features.dim() > 2:
            query_features = F.adaptive_avg_pool2d(query_features, 1).view(query_features.size(0), -1)

        # Align device/dtype with queue buffer
        query_features = query_features.to(self.queue.device).to(self.queue.dtype)
        return self.classifier(query_features)

    def update_momentum(self, momentum=0.999):
        update_momentum_encoder(self.encoder_q, self.encoder_k, momentum)


@torch.no_grad()
def update_momentum_encoder(encoder_q, encoder_k, momentum=0.999):
    for param_q, param_k in zip(encoder_q.parameters(), encoder_k.parameters()):
        param_k.data = param_k.data * momentum + param_q.data * (1. - momentum)


def moco_forward(encoder_q, encoder_k, queue, x):
    # Forward through query encoder (with grads)
    query_features = encoder_q(x)

    # Forward through momentum encoder (no grads, inference mode)
    with torch.inference_mode():
        # Ensure x is on the same device as encoder_k
        x_k = x.to(next(encoder_k.parameters()).device, non_blocking=True)
        mom_features = encoder_k(x_k)
        # Move momentum features to queue device/dtype for concatenation
        mom_features = mom_features.to(queue.device).to(queue.dtype)

    # Update queue: prepend mom_features, keep fixed size
    queue = torch.cat([mom_features, queue], dim=0)[: queue.shape[0]]
    return query_features, mom_features, queue


# === Model builders ===
def make_resnet18(num_class):
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    model.fc = nn.Sequential(
        nn.Dropout(p=0.5),
        nn.Linear(model.fc.in_features, num_class)
    )
    return model


def make_efficientnet_b3(num_class):
    model = timm.create_model("efficientnet_b3", pretrained=True)
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.5),
        nn.Linear(model.classifier.in_features, num_class)
    )
    return model


def build_moco_effcombo_model(num_classes, queue_size=32, alpha=0.5, img_size=384):
    # Create two EfficientNet-B0 encoders
    encoder_q = timm.create_model("efficientnet_b0", pretrained=True)
    encoder_q.classifier = nn.Identity()

    encoder_k = timm.create_model("efficientnet_b0", pretrained=True)
    encoder_k.classifier = nn.Identity()
    encoder_k.eval()  # momentum encoder stays in eval

    # Infer embedding dim using intended input resolution
    dummy_input = torch.randn(1, 3, img_size, img_size)
    with torch.no_grad():
        dummy_output = encoder_q(dummy_input)
        if dummy_output.dim() > 2:
            dummy_output = F.adaptive_avg_pool2d(dummy_output, 1).view(1, -1)
        embedding_dim = dummy_output.shape[1]

    # Initialize queue (will be registered as buffer and moved with model.to(device))
    queue = F.normalize(torch.randn(queue_size, embedding_dim), dim=1)

    model = MoCoEffComboModel(
        encoder_q=encoder_q,
        encoder_k=encoder_k,
        queue=queue,
        embedding_dim=embedding_dim,
        alpha=alpha,
        num_class=num_classes
    )
    return model


# === Training setup ===
models_to_train = [
    ("EfficientNetB3", lambda num_classes: make_efficientnet_b3(num_classes)),
    # ("ResNet18", lambda num_classes: make_resnet18(num_classes)),
    ("MoCoEffCombo", lambda num_classes: build_moco_effcombo_model(num_classes, queue_size=32, alpha=0.5, img_size=384))
]


In [ ]:
num_classes = len(classes)
print(num_classes)

8


In [ ]:
# Maybe add scheduler
from torch.optim.lr_scheduler import CosineAnnealingLR

In [ ]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)


In [ ]:
import os
from torch.utils.data import Subset, DataLoader, WeightedRandomSampler
from collections import Counter
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import gc


for model_name, model_fn in models_to_train:
    all_fold_results = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(np.arange(len(dataset)), dataset.targets)):
        print(f"\n===== {model_name} | Fold {fold+1}/{n_splits} =====")

        # --- Fold-specific file paths ---
        checkpoint_path = os.path.join(project_root, f"{model_name}_fold{fold+1}_checkpoint_latest.pth")
        train_val_path = os.path.join(project_root, f"{model_name}_fold{fold+1}_train_val_log.csv")
        test_results_path = os.path.join(project_root, f"{model_name}_fold{fold+1}_test_results.csv")

        # --- Skip if results already exist ---
        if os.path.exists(test_results_path):
            print(f"Skipping fold {fold+1} — results already exist")
            # Optionally, load the existing results into all_fold_results
            existing_results = pd.read_csv(test_results_path)
            all_fold_results.append(existing_results)
            continue

        torch.cuda.empty_cache()

        # --- Fresh model for this fold ---
        model = model_fn(num_classes).to(device)


        # --- Optimizer & scheduler ---
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        scheduler = CosineAnnealingLR(optimizer, T_max=30, eta_min=1e-6)

        # --- Build fold datasets ---
        train_set = Subset(dataset, train_idx)
        val_set   = Subset(dataset, val_idx)

        # Apply transforms
        train_set.dataset.transform = AlbTransform(train_augmentations)
        val_set.dataset.transform = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        # --- Compute fold-specific class weights ---
        class_counts = Counter([dataset.targets[i] for i in train_idx])
        weights = [1.0 / class_counts[c] for c in range(len(classes))]
        weights_tensor = torch.tensor(weights, dtype=torch.float32).to(device)

        # --- Weighted sampler ---
        sample_weights = [weights[label] for label in [dataset.targets[i] for i in train_idx]]
        sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

        # --- DataLoaders ---
        globals()['train_loader'] = DataLoader(train_set, batch_size=16, sampler=sampler)
        globals()['val_loader']   = DataLoader(val_set, batch_size=16, shuffle=False)

        # --- Train this fold ---
        run_training(
            model_name=f"{model_name}_fold{fold+1}",
            model=model,
            optimizer=optimizer,
            scheduler=scheduler,
            train_val_path=train_val_path,
            test_results_path=test_results_path,
            checkpoint_path=checkpoint_path,
            num_epochs=30
        )

        # --- Collect fold results ---
        fold_results = pd.read_csv(test_results_path)
        all_fold_results.append(fold_results)

        # Cleanup
        del model, optimizer
        torch.cuda.empty_cache()
        gc.collect()

    # === Average results across folds ===
    avg_results = pd.concat(all_fold_results).mean(numeric_only=True)
    avg_results["Model"] = model_name
    avg_results.to_csv(os.path.join(project_root, f"{model_name}_cv_average_results.csv"), index=False)
    print(f"\n{model_name} CV average results:\n", avg_results)



===== EfficientNetB3 | Fold 1/5 =====
Skipping fold 1 — results already exist

===== EfficientNetB3 | Fold 2/5 =====
Skipping fold 2 — results already exist

===== EfficientNetB3 | Fold 3/5 =====
Skipping fold 3 — results already exist

===== EfficientNetB3 | Fold 4/5 =====
Skipping fold 4 — results already exist

===== EfficientNetB3 | Fold 5/5 =====
Skipping fold 5 — results already exist

EfficientNetB3 CV average results:
 Test Loss               0.001232
Test Accuracy           0.446752
Test Precision          0.416494
Test Recall             0.446752
Test F1                 0.412846
Test AUC                0.842988
Model             EfficientNetB3
dtype: object

===== MoCoEffCombo | Fold 1/5 =====
Skipping fold 1 — results already exist

===== MoCoEffCombo | Fold 2/5 =====
Skipping fold 2 — results already exist

===== MoCoEffCombo | Fold 3/5 =====
Skipping fold 3 — results already exist

===== MoCoEffCombo | Fold 4/5 =====
Skipping fold 4 — results already exist

===== MoCoEffC